In [10]:
import requests
from bs4 import BeautifulSoup
import os, re
from openai import OpenAI
import time  
from datetime import datetime  

# 'https://menet.com.cn

In [13]:
host ='https://www.menet.com.cn'

In [15]:
def get_list():
    host ='https://www.menet.com.cn/news'
    response = requests.post(host)
    soup = BeautifulSoup(response.text, 'html.parser')
    news_list = soup.find_all('h3')
    return soup, news_list

In [17]:
soup, news_list = get_list()

In [19]:
news_list = [_ for _ in news_list if "登录" not in _.text]

In [21]:
def get_info(item, host ='https://www.menet.com.cn'):
    link = host + item.a['href']
    title = item.text
    date = item.a['href'].split('/')[3][:8]
    return {
    "title": title,
    'url': link,
    'date':date
}

In [26]:
result1 = get_info))

SyntaxError: unmatched ')' (2504085509.py, line 1)

In [24]:
for num, item in enumerate(soup.find_all(lambda tag: tag.has_attr('class') and len(tag)==1 and tag['class'] == ["item_time"])):
    result1[num]['date'] = item.text

NameError: name 'result1' is not defined

# https://www.pharnexcloud.com/zixun

In [18]:
r = requests.get('https://www.pharnexcloud.com/zixun')

In [19]:
soup = BeautifulSoup(r.text, 'html.parser')

In [20]:
dates = soup.find_all(lambda tag:tag.has_attr('class') and tag['class']==['date'])

In [22]:
result = soup.find_all('a', class_ = 'img-content')

In [28]:
def query_tilte_url(item):
    title = item.find_next()['alt']
    url = 'https://www.pharnexcloud.com' + item['href']
    return {
        'title': title,
        'url': url
    }

In [30]:
res = list(map(query_tilte_url, result))

In [32]:
for no, item in enumerate(res):
    
    item['date'] = dates[no]

In [34]:
result2 = res

---

# https://bydrug.pharmcube.com/

In [8]:
r = requests.get('https://bydrug.pharmcube.com/')

In [9]:
soup = BeautifulSoup(r.text, 'html.parser')

In [10]:
datas = soup.find_all('div', class_= 'mf-flex mf-items-center')

In [14]:
def get_tilte_and_url(item):
    title = item.text.strip()
    url = item.a['href']
    return {
        'title': title,
        'url': url
    }

In [16]:
res = list(map(get_tilte_and_url, datas))

TypeError: 'NoneType' object is not subscriptable

In [50]:
for no, _ in enumerate(res):
    _['date'] = dates[no]

In [52]:
result3 = res

In [54]:
import pandas as pd

In [56]:
df = pd.DataFrame(result1+result2+result3)

---

In [60]:
# 用r.jina转换成ai友好格式，并用通义千问进行总结

In [66]:
def summarize(url):
    context = requests.get('https://r.jina.ai/'+url).text
    url_pattern = re.compile(r'\bhttps?:\/\/[^\s/$.?#].[^\s]*\b')  
    cleaned_text = url_pattern.sub('', context)
    context = cleaned_text[:6000] if len(cleaned_text)>6000  else cleaned_text
    context = "请尽量简洁总结以下内容，并发表你的观点,文字不得超过150字\n"+context.replace("\n","")[:6000]
    client = OpenAI(
    api_key='sk-69c9bf69fa2b4d2985834d6dc17c0790', # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
    )
    completion = client.chat.completions.create(
        model="qwen-max", # 更多模型请参见模型列表文档。
        messages=[{"role": "user", "content": context}]
    )
    print(completion.choices[0].message.content)
    with open('output.md', 'w') as f:  
        # 使用 print 函数，将输出重定向到文件  
        print(completion.choices[0].message.content, file=f)  
        
    return completion.choices[0].message.content

In [68]:
df

,title,url,date
0,“数智引领 价值重构”——2024全国慢病管理零售论坛顺利召开,https://www.menet.com.cn/info/202409/202409091...,2024-09-09
1,山东新时代爆发了！拿下9个重磅品种，猛攻1100亿市场，11款1类新药、44个新品蓄力,https://www.menet.com.cn/info/202409/202409051...,2024-09-05
2,492只医药股上半年成绩单！迈瑞净利润首破70亿，5家晋身10亿军团，甘李、海思科……翻倍,https://www.menet.com.cn/info/202409/202409051...,2024-09-05
3,63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏,https://www.menet.com.cn/info/202409/202409050...,2024-09-05
4,信立泰厉害了！12款1类新药曝光，猛攻1300亿市场，5款新药上市可期,https://www.menet.com.cn/info/202409/202409031...,2024-09-03
...,...,...,...
181,医药与健康护理行业周报：2024年1-7月医保统筹基金主要指标发布,https://bydrug.pharmcube.com/report/detail/b0a...,2024-09-09 10:51:50
182,医药行业周报：武田1类创新药拟纳入突破性治疗品种,https://bydrug.pharmcube.com/report/detail/bbd...,2024-09-09 10:41:53
183,生物医药行业周报：高值耗材集采进入密集执行期，建议关注相关优质企业份额提升机会,https://bydrug.pharmcube.com/report/detail/796...,2024-09-09 10:23:50
184,医药生物行业报告：半年报业绩整体稳健，分化中挖掘底部机会,https://bydrug.pharmcube.com/report/detail/ef3...,2024-09-09 10:02:16


In [70]:
result = result1 + result2 + result3

In [72]:
def get_and_send(item):
    url = item['url']
    title= item['title']
    summary = summarize(url)
    return summary

In [74]:
li = result = result1 + result2 + result3

In [76]:
li[3]

{'title': '63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏',
 'url': 'https://www.menet.com.cn/info/202409/202409050941374137_148059.shtml',
 'date': '2024-09-05'}

In [150]:
title, url, date = li[3].values()

In [140]:
title

'63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏'

In [84]:
url

'https://www.menet.com.cn/info/202409/202409050941374137_148059.shtml'

In [86]:
date

'2024-09-05'

In [411]:
l = li[:10]

In [168]:
summary = get_and_send(li[3])

2024年Q1中国零售药店中成药市场强劲，销售额超390亿元，63个品种销售额破亿，主要集中在呼吸系统治疗。这些药品总销售额超160亿，其中3个品种销售额超10亿，增长迅猛，9个品种增长率超50%，2个品种增长率甚至达到133%。市场亮点还包括34个独家品种的突出表现，云南白药等企业成为领军者。此数据反映出中成药市场需求旺盛，独家品种优势显著，行业巨头表现抢眼。


In [457]:
l

[{'title': '“数智引领 价值重构”——2024全国慢病管理零售论坛顺利召开',
  'url': 'https://www.menet.com.cn/info/202409/202409091619471947_148075.shtml',
  'date': '2024-09-09'},
 {'title': '山东新时代爆发了！拿下9个重磅品种，猛攻1100亿市场，11款1类新药、44个新品蓄力',
  'url': 'https://www.menet.com.cn/info/202409/20240905102904294_148064.shtml',
  'date': '2024-09-05'},
 {'title': '492只医药股上半年成绩单！迈瑞净利润首破70亿，5家晋身10亿军团，甘李、海思科……翻倍',
  'url': 'https://www.menet.com.cn/info/202409/202409051027242724_148062.shtml',
  'date': '2024-09-05'},
 {'title': '63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏',
  'url': 'https://www.menet.com.cn/info/202409/202409050941374137_148059.shtml',
  'date': '2024-09-05'},
 {'title': '信立泰厉害了！12款1类新药曝光，猛攻1300亿市场，5款新药上市可期',
  'url': 'https://www.menet.com.cn/info/202409/20240903141804184_148053.shtml',
  'date': '2024-09-03'},
 {'title': '“中药股王”发威！肝病用药狂卖26亿，14个独家中成药热销，8款重磅新药出击',
  'url': 'https://www.menet.com.cn/info/202409/202409031416261626_148051.shtml',
  'date': '2024-09-03'},
 {'title': '第十批集采24个品种备战！千亿市场齐鲁亮了，2大注射剂疯

In [459]:
for k, v in l[0].items():
    print(k, v)

title “数智引领 价值重构”——2024全国慢病管理零售论坛顺利召开
url https://www.menet.com.cn/info/202409/202409091619471947_148075.shtml
date 2024-09-09


In [616]:
no = [str(_)+'.' for _ in range(1, 11)] 

In [680]:
li = [f"[{_['title']}]({_['url']})" for _ in l ]

In [682]:
li = [b[:1] + a+ b[1:] for a, b in zip(no, li)]

In [684]:
li

['[1.“数智引领 价值重构”——2024全国慢病管理零售论坛顺利召开](https://www.menet.com.cn/info/202409/202409091619471947_148075.shtml)',
 '[2.山东新时代爆发了！拿下9个重磅品种，猛攻1100亿市场，11款1类新药、44个新品蓄力](https://www.menet.com.cn/info/202409/20240905102904294_148064.shtml)',
 '[3.492只医药股上半年成绩单！迈瑞净利润首破70亿，5家晋身10亿军团，甘李、海思科……翻倍](https://www.menet.com.cn/info/202409/202409051027242724_148062.shtml)',
 '[4.63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏](https://www.menet.com.cn/info/202409/202409050941374137_148059.shtml)',
 '[5.信立泰厉害了！12款1类新药曝光，猛攻1300亿市场，5款新药上市可期](https://www.menet.com.cn/info/202409/20240903141804184_148053.shtml)',
 '[6.“中药股王”发威！肝病用药狂卖26亿，14个独家中成药热销，8款重磅新药出击](https://www.menet.com.cn/info/202409/202409031416261626_148051.shtml)',
 '[7.第十批集采24个品种备战！千亿市场齐鲁亮了，2大注射剂疯狂连涨，倍特、华海蓄力](https://www.menet.com.cn/info/202409/20240903141404144_148049.shtml)',
 '[8.28款儿科中成药亮眼！葵花、健民、济川领跑实体药店，3大品牌网上药店翻倍](https://www.menet.com.cn/info/202409/202409031411121112_148047.shtml)',
 '[9.翰森制药2024上半年营收大涨！净利润翻番！](https://www.menet.com.cn/info/2

In [686]:
li = [_ + '\n\n' for _ in li ]

In [688]:
li

['[1.“数智引领 价值重构”——2024全国慢病管理零售论坛顺利召开](https://www.menet.com.cn/info/202409/202409091619471947_148075.shtml)\n\n',
 '[2.山东新时代爆发了！拿下9个重磅品种，猛攻1100亿市场，11款1类新药、44个新品蓄力](https://www.menet.com.cn/info/202409/20240905102904294_148064.shtml)\n\n',
 '[3.492只医药股上半年成绩单！迈瑞净利润首破70亿，5家晋身10亿军团，甘李、海思科……翻倍](https://www.menet.com.cn/info/202409/202409051027242724_148062.shtml)\n\n',
 '[4.63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏](https://www.menet.com.cn/info/202409/202409050941374137_148059.shtml)\n\n',
 '[5.信立泰厉害了！12款1类新药曝光，猛攻1300亿市场，5款新药上市可期](https://www.menet.com.cn/info/202409/20240903141804184_148053.shtml)\n\n',
 '[6.“中药股王”发威！肝病用药狂卖26亿，14个独家中成药热销，8款重磅新药出击](https://www.menet.com.cn/info/202409/202409031416261626_148051.shtml)\n\n',
 '[7.第十批集采24个品种备战！千亿市场齐鲁亮了，2大注射剂疯狂连涨，倍特、华海蓄力](https://www.menet.com.cn/info/202409/20240903141404144_148049.shtml)\n\n',
 '[8.28款儿科中成药亮眼！葵花、健民、济川领跑实体药店，3大品牌网上药店翻倍](https://www.menet.com.cn/info/202409/202409031411121112_148047.shtml)\n\n',
 '[9.翰森制药2024上半年营收大涨！净利润翻番！]

In [670]:
url2 = 'https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=660ed8dd-be77-4f1b-8de8-774d53f1f39c'

In [637]:
send_markdown_to_wechat(f"# [{title}]({url})", )

In [638]:
send_markdown_to_wechat(f"### [{title}]({url})", url )

In [180]:
link = f"### [{title}]({url})"

In [391]:
title

'63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏'

In [172]:
link+= '/n'

In [405]:
link

'### [63个中成药厉害了！独家品种大涨133%，3个超10亿品种亮眼，扬子江、济川、天士力霸屏](https://www.menet.com.cn/info/202409/202409050941374137_148059.shtml)'

'2024年Q1中国零售药店中成药市场强劲，销售额超390亿元，63个品种销售额破亿，主要集中在呼吸系统治疗。这些药品总销售额超160亿，其中3个品种销售额超10亿，增长迅猛，9个品种增长率超50%，2个品种增长率甚至达到133%。市场亮点还包括34个独家品种的突出表现，云南白药等企业成为领军者。此数据反映出中成药市场需求旺盛，独家品种优势显著，行业巨头表现抢眼。'

In [242]:
send_card_to_wechat()

In [401]:
def send_markdown_to_wechat(msg='test', webhook_url='https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=a0ad9b92-b9a5-43d3-a129-98c954ecb159'):
    
    headers = {  
        'Content-Type': 'application/json',  
    }

    data = dic
   
    response = requests.post(webhook_url, json=data, headers=headers)

In [672]:
send_markdown_to_wechat(msg = "".join(li))

In [690]:
send_markdown_to_wechat(msg = "".join(li), webhook_url='https://qyapi.weixin.qq.com/cgi-bin/webhook/send?key=660ed8dd-be77-4f1b-8de8-774d53f1f39c')

In [ ]:
s